In [1]:
import yaml

full_text = ''

with open("./data/processed/Agentic_Design_Patterns.mmd", "r") as f:
    full_text = f.read()

In [11]:
def split_into_four_parts(text: str):
    length = len(text)
    part_size = length // 4  # integer division

    parts = [
        text[i * part_size : (i + 1) * part_size]
        for i in range(3)  # first 3 parts
    ]

    # last part takes the remainder
    parts.append(text[3 * part_size:])

    return parts


# Example
parts = split_into_four_parts(full_text)

for i, p in enumerate(parts, start=1):
    print(f"Part {i}: {len(p)} characters")

Part 1: 214275 characters
Part 2: 214275 characters
Part 3: 214275 characters
Part 4: 214278 characters


In [16]:
with open("./prompts/rag_validation_openai.yml") as f:
    prompts = yaml.safe_load(f)

system_prompt = prompts["rag_validation_prompt"]["system_template"].format(
    text_book=p
)
user_prompt   = prompts["rag_validation_prompt"]["user_template"].format(
    num_questions=10,
    role="student",
)

In [17]:
print(system_prompt)

You are an expert synthetic data generator for evaluating Retrieval-Augmented Generation (RAG) systems built on top of technical books.
Your job is to produce high-quality, deeply grounded, human-sounding technical questions and their correct answers.

BOOK PASSAGE (Use this as your ONLY source):
  tterns.- Example: Classify the sentiment of the following movie reviews as POSITIVE, NEUTRAL, or




NEGATIVE:  


Review: "The acting was superb and the story was engaging." Sentiment: POSITIVE  


Review: "It was okay, nothing special." Sentiment: NEUTRAL  


Review: "I found the plot confusing and the characters unlikable." Sentiment: NEGATIVE  


Review: "The visuals were stunning, but the dialogue was weak." Sentiment:  


Understanding when to apply zero- shot, one- shot, and few- shot prompting techniques, and thoughtfully crafting and organizing examples, are essential for enhancing the effectiveness of agentic systems. These basic methods serve as the groundwork for various promptin

In [18]:
print(user_prompt)

Generate 10 question–answer pairs based on the following book passage.
Assume the person asking the questions has the following persona: student.


In [ ]:
roles = [
     "computer science student", 
     "Junior AI/ML Engineer", 
     "Senior AI/ML Engineer or Architect",
     "Researcher in AI / Autonomous Systems",
     "Product Manager (AI-focused)",
     "Software Engineer Exploring AI Adoption"
]

In [20]:
from pydantic import BaseModel, Field
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

class QuestionAnswer(BaseModel):
    question: str
    answer: str
    
class QAList(BaseModel):
    q_a_pairs: list[QuestionAnswer]
    
    
response = client.responses.parse(
    model="gpt-4o-mini",
    input=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    text_format=QAList
)

qa_pairs = response.output_parsed

In [24]:
qa_pairs.q_a_pairs[0].question, qa_pairs.q_a_pairs[0].answer

('What is zero-shot prompting, and when should it be used?',
 "Zero-shot prompting involves providing a prompt without any examples, relying solely on the model's pre-existing knowledge to generate a response. It is typically used when a user expects the model to understand and respond to a request without needing prior illustrations.")